<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/ChromaticityPlotterV1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install colormath

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from PIL import Image
import requests
from io import BytesIO
from colormath.color_objects import sRGBColor, LabColor, XYZColor
from colormath.color_conversions import convert_color
import matplotlib.patches as patches
from google.colab import files

# Step 1: Load your LAB color dataset with Getty AAT names
# You can upload your CSV file containing 'L', 'A', 'B', and 'Color Name' columns

# Upload the CSV file
print("Please upload your LAB color dataset CSV file.")
uploaded_files = files.upload()

# Get the CSV file name
csv_filename = next(iter(uploaded_files))

# Load the dataset
dataset = pd.read_csv(csv_filename)

# Display the first few rows
print("First few rows of your dataset:")
print(dataset.head())

# Check the column names
print("\nColumn names in your dataset:")
print(dataset.columns.tolist())

# Check data types
print("\nData types of each column:")
print(dataset.dtypes)

# Check for NaN values
print("\nChecking for NaN values in 'L', 'A', 'B' columns:")
print(dataset[['L', 'A', 'B']].isnull().sum())

# Check for infinite values
print("\nChecking for infinite values in 'L', 'A', 'B' columns:")
print(np.isinf(dataset[['L', 'A', 'B']]).sum())

# Convert 'L', 'A', 'B' columns to numeric, coercing errors to NaN
dataset[['L', 'A', 'B']] = dataset[['L', 'A', 'B']].apply(pd.to_numeric, errors='coerce')

# Replace infinite values with NaN
dataset[['L', 'A', 'B']] = dataset[['L', 'A', 'B']].replace([np.inf, -np.inf], np.nan)

# Drop rows with NaN values in 'L', 'A', 'B'
dataset = dataset.dropna(subset=['L', 'A', 'B'])

# Re-check for NaN values after cleaning
print("\nAfter cleaning, checking for NaN values again:")
print(dataset[['L', 'A', 'B']].isnull().sum())

# Step 1 (continued): Build the KDTree
lab_values = dataset[['L', 'A', 'B']].values  # Get the LAB values from the dataset
lab_tree = KDTree(lab_values)  # Build the KDTree from these LAB values

# Step 2: Load the image to analyze
# Option 1: Upload an image file
print("\nPlease upload the image file you want to analyze.")
uploaded_images = files.upload()

# Get the image file name
image_filename = next(iter(uploaded_images))

# Open the image
image = Image.open(image_filename).convert('RGB')

# Step 3: Display the original image
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.axis('off')
plt.title('Original Image')
plt.show()

# Step 4: Define the Region of Interest (ROI)
# You can analyze the whole image or specify a ROI

# Option 1: Use the whole image
roi = image

# Step 5: Extract the average color from the ROI
# Convert ROI to a NumPy array
roi_array = np.array(roi)

# Calculate the average color
average_color = roi_array.mean(axis=(0, 1))
average_color_normalized = average_color / 255.0  # Normalize RGB values to [0, 1]

# Display the average color patch
average_color_patch = np.ones((100, 100, 3)) * average_color_normalized
plt.figure(figsize=(2, 2))
plt.imshow(average_color_patch)
plt.axis('off')
plt.title('Average Color Extracted')
plt.show()

# Step 6: Convert the average RGB color to LAB color space
avg_r, avg_g, avg_b = average_color_normalized
rgb_color = sRGBColor(avg_r, avg_g, avg_b)
lab_color = convert_color(rgb_color, LabColor)
L = lab_color.lab_l
A = lab_color.lab_a
B = lab_color.lab_b

# Step 7: Find the five closest colors in your dataset
distance, indices = lab_tree.query([L, A, B], k=5)  # Find the 5 closest colors
closest_colors = dataset.iloc[indices]  # Fetch the rows of the 5 closest colors

# Step 8: Convert the closest LAB colors back to RGB for display
fig, axs = plt.subplots(1, 5, figsize=(15, 3))  # Create subplots for 5 colors

for i, (index, dist) in enumerate(zip(indices, distance)):
    closest_color = dataset.iloc[index]
    closest_color_name = closest_color['Color Name']

    lab_color_closest = LabColor(closest_color['L'], closest_color['A'], closest_color['B'])
    rgb_color_closest = convert_color(lab_color_closest, sRGBColor)

    # Ensure RGB values are within [0, 1]
    rgb_values_closest = [max(0, min(1, val)) for val in [rgb_color_closest.rgb_r, rgb_color_closest.rgb_g, rgb_color_closest.rgb_b]]

    # Display the closest matching color patch
    closest_color_patch = np.ones((100, 100, 3)) * rgb_values_closest
    axs[i].imshow(closest_color_patch)
    axs[i].axis('off')
    axs[i].set_title(f"{closest_color_name}\nDistance: {dist:.2f}")

plt.show()

# Step 9: Convert the RGB color to XYZ and calculate chromaticity coordinates for all 5 colors
chromaticity_coords = []
for i, index in enumerate(indices):
    closest_color = dataset.iloc[index]
    lab_color_closest = LabColor(closest_color['L'], closest_color['A'], closest_color['B'])
    rgb_color_closest = convert_color(lab_color_closest, sRGBColor)

    # Convert RGB to XYZ to get chromaticity coordinates
    xyz_color = convert_color(rgb_color_closest, XYZColor)
    X, Y, Z = xyz_color.xyz_x, xyz_color.xyz_y, xyz_color.xyz_z
    total = X + Y + Z
    x_chroma = X / total if total != 0 else 0
    y_chroma = Y / total if total != 0 else 0

    chromaticity_coords.append((x_chroma, y_chroma))

# Step 10: Upload and use a local PNG version of the chromaticity diagram
print("Please upload a PNG file of the CIE 1931 Chromaticity Diagram.")
uploaded_files = files.upload()

# Get the file name for the chromaticity diagram
cie_diagram_filename = next(iter(uploaded_files))

# Step 11: Plot the chromaticity coordinates on the uploaded CIE 1931 diagram and save the plot
def plot_chromaticity_diagram(coords, cie_diagram_filename):
    cie_diagram = plt.imread(cie_diagram_filename)

    plt.figure(figsize=(8, 6))
    plt.imshow(cie_diagram, extent=(0, 0.8, 0, 0.9))

    # Plot all chromaticity coordinates
    for (x, y) in coords:
        plt.scatter(x, y, color='red', edgecolors='black', zorder=5)

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('CIE 1931 Chromaticity Diagram with 5 Closest Colors')

    # Save the plot as a PNG file
    diagram_filename = 'chromaticity_diagram.png'
    plt.savefig(diagram_filename, dpi=300, bbox_inches='tight')
    plt.show()

    # Allow the user to download the saved diagram
    files.download(diagram_filename)

# Plot the chromaticity coordinates for all 5 colors and save the image
plot_chromaticity_diagram(chromaticity_coords, cie_diagram_filename)

# Step 12: Display the results for each of the 5 colors
for i, (index, dist) in enumerate(zip(indices, distance)):
    closest_color = dataset.iloc[index]
    print(f"Color {i+1}:")
    print(f"Closest LAB in Dataset: L = {closest_color['L']:.2f}, a = {closest_color['A']:.2f}, b = {closest_color['B']:.2f}")
    print(f"Distance in LAB Color Space: {dist:.2f}")
    print(f"Color Name: {closest_color['Color Name']}\n")